# ResNet CIFAR-10 with tensorboard

このノートブックでは、TensorBoardの使用方法と、トレーニングジョブがチェックポイントを外部バケットに書き込む方法を示します。
このノートブックで使用されるモデルは、CIFAR-10データセットで訓練されたResNetモデルです。より多くの背景については、以下の論文を参照してください。

[Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf) by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Dec 2015.

[Identity Mappings in Deep Residual Networks](https://arxiv.org/pdf/1603.05027.pdf) by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Jul 2016.

### 環境設定

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

### CIFAR-10 データセットのダウンロード
テストデータとトレーニングデータのダウンロードには約5分かかります。

In [ ]:
import utils

utils.cifar10_download()

### データをS3 bucketにアップロードする

In [ ]:
inputs = sagemaker_session.upload_data(path='/tmp/cifar10_data', key_prefix='data/DEMO-cifar10')

**sagemaker_session.upload_data** はあなたのマシンからCIFAR-10データセットを **sagemaker-{region}-{*your aws account number*}** というバケットにアップロードします。もしあなたがまだこのバケットを持っていなければ、 sagemaker_sessionがそれを作成します。

### モデルのソースコードとスクリプト
- [source_dir/resnet_model.py](source_dir/resnet_model.py): ResNetモデル
- [source_dir/resnet_cifar_10.py](source_dir/resnet_cifar_10.py): 学習とホスティングのためのメインスクリプト

## sagemaker.TensorFlow estimatorを使ってトレーニングジョブを生成

In [ ]:
from sagemaker.tensorflow import TensorFlow


source_dir = os.path.join(os.getcwd(), 'source_dir')
estimator = TensorFlow(entry_point='resnet_cifar_10.py',
                       source_dir=source_dir,
                       role=role,
                       framework_version='1.11.0',
                       hyperparameters={'throttle_secs': 30},
                       training_steps=1000, evaluation_steps=100,
                       train_instance_count=4, train_instance_type='ml.c5.2xlarge', 
                       base_job_name='tensorboard-example')

%time estimator.fit(inputs, run_tensorboard_locally=True)

---
**```fit```** メソッドは2つの **ml.c5.2xlarge** インスタンスで **```tensorboard-example- {一意の識別子}```** という名前のトレーニングジョブを作成します。
これらのインスタンスは、s3バケット **```sagemaker- {あなたのawsアカウント番号}```** にチェックポイントを書き出します。

まだこのバケットがない場合、** ```sagemaker_session``` **があなたのために作成します。これらのチェックポイントは、トレーニングジョブを復元したり、
**TensorBoard**を使用してトレーニングジョブのメトリクスを分析するために使用できます。

パラメータ **```run_tensorboard_locally = True```** は、このノートブックが動作しているマシンで **TensorBoard** を実行します。
S3バケットのトレーニングジョブによって新しいチェックポイントが作成されるたびに、 **```fit```** は **TensorBoard**が指している一時フォルダにチェックポイントをダウンロードします。

** ```fit```** メソッドがトレーニングを開始すると、** TensorBoard** がその指標を表示するために使用しているポートを記録します。
デフォルトのポートは**6006**ですが、可用性に応じて別のポートを選択できます。ポート番号は使用可能なポートが見つかるまで増加します。その後、ポート番号がstdoutに出力されます。

コンテナをプロビジョニングしてトレーニングを開始するには数分かかります**TensorBoard**はすぐ後に測定基準を表示し始めます。

[http：//localhost：6006](http：//localhost：6006)またはSageMakerノートブックインスタンス+[/proxy/6006/](/proxy/6006/)を使用して、
**TensorBoard** にローカルでアクセスできます (TensorBoardは、URLの最後にスラッシュを置くのを忘れた場合は機能しません）。
TensorBoardが別のポートで起動した場合は、これらのURLを一致させるように調整します。
この例では、オプションのハイパーパラメータ **```throttle_secs```** を使用して、トレーニング評価をより頻繁に生成し、**TensorBoard**
スカラーデータをより速く視覚化できます。[利用可能なオプションのハイパーパラメータ](https://github.com/aws/sagemaker-python-sdk#optional-hyperparameters)があります。

---

## 訓練されたモデルを展開して予測を準備する

deploy（）メソッドにより、予測リクエストをリアルタイムで処理するエンドポイントを作成します。

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## フェイクデータで予測して、エンドポイントがアップしていることを確認する

予測はこのノートブックの焦点ではないので、エンドポイントの機能を確認するために、正しいフォーマットのランダムデータを生成して予測を行います。

In [ ]:
import numpy as np

random_image_data = np.random.rand(32, 32, 3)
predictor.predict(random_image_data)

# クリーンアップ
このチュートリアルで使用されているリソースのAWSアカウントへの課金を避けるには、**SageMaker Endpoint** を削除します。

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)